# Common Libraries

In [15]:
# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

from sklearn.decomposition import PCA, KernelPCA
import util

# Load Dataset


Read the date, and 8 yields (1,2,3,5,10,20,30 year) in to a data frame.
Then filter out any rows with null values for any of the above 8 yields.
Make the Date the index in this input data frame.
Find, the end of month dates for the year range of the input data frame.
Filter and keep only the rows for the end of the month date.


In [16]:
%matplotlib inline
df = pd.read_csv('yield_curves.csv', sep=' *, *')
df = df.replace({'na': np.nan})
df = df.apply(pd.to_numeric, errors='ignore')

# Remove the last column due to the last comma
df = df.iloc[:, :-1]
df['Date'] = pd.to_datetime(df['Date'],infer_datetime_format=True)
df.set_index('Date', drop=True, inplace=True)

# drop na rows
df = df.dropna()

min_year = df.index.min().year
max_year = df.index.max().year

last_day_of_month = util.lastdayofmonth(min_year, max_year)
last_day_of_month.set_index('Date', drop=True, inplace=True)

i1 = df.index
i2 = last_day_of_month.index
df_monthly_samples = df[i1.isin(i2)]

df_selected_yields = df_monthly_samples.loc[:,["ZC100YR", "ZC200YR", "ZC300YR", "ZC500YR", "ZC1000YR", "ZC1500YR", "ZC2000YR", "ZC3000YR"]]
df_selected_yields.columns = ["1YR", "2YR", "3YR", "5YR", "10YR", "15YR", "20YR", "30YR"]

# Projection onto Principal Components
Use Kernel PCA for better accuracy

# Kernel PCA

In [17]:
pcaA = KernelPCA(n_components=3,
                 kernel='rbf',
                 gamma=4,
                 kernel_params=None,
                 fit_inverse_transform=False,
                 eigen_solver='auto',
                 tol=0,
                 max_iter=None)

pcaA.fit(df_selected_yields)
dpca = pd.DataFrame(pcaA.transform(df_selected_yields))
dpca.index = df_selected_yields.index

The objective is to model how yield curve move with the changes in PCA 1.
The alogrithm (later extend to multiple output ANFIS) is using single output ANFIS and calculate each yield seperately. Therefore the error function is not optimal as with all outputs together.
For the moment, only 5 year yield is predicted.


The current state is represented by PCA. 
Change in PCA 1 is calculated by substracting from the next month PCA 1.
Expected output is change in the yield curve.


ANFIS Model
ANFIS  inputs = PC1 (t), PC2 (t), Delta PCA 1 (t + delta t)
ANFIS  output = Delta Y (t + delta t)
Each input is represented by 4 gaussian membership functions.

TODO
Modify the alogrithm to train the 8 yields together, so the ANFIS is generalized for mutliple yields
ANFIS  inputs = PC1 (t), PC2 (t), Delta PCA 1 (t + delta t)
ANFIS  output = Delta Y1 (t + delta t), Delta Y2 (t + delta t) ... Delta Y8 (t + delta t)



In [18]:
delta_pca = dpca.diff()
df_input = pd.merge(dpca, delta_pca, on='Date')

df_selected_yields_delta = df_selected_yields.diff()

Import the ANFIS classes
Copied from https://github.com/twmeggs/anfis
Will extensively modify/extend and send a pull request

In [19]:
import anfis
import mfDerivs
import membershipfunction

Convert the data frames to numpy arrays as expected by the ANFIS library

In [20]:
# df_input contains PC1, PC2, and Delta PC1
ts = df_input.to_numpy()
X = ts[1:, 0:3]

df_selected_yields_delta_filtered = df_selected_yields_delta.iloc[1:, 0:]
Y = np.stack(df_selected_yields_delta_filtered.to_numpy())


Define the membership functions. Four per each variable, PC1, PC2, Delta PC1

In [21]:
import os

mf = {}
mf["gauss_4"] = [
    [['gaussmf',{'mean':0.2,'sigma':0.1}],['gaussmf',{'mean':0.4,'sigma':0.1}],['gaussmf',{'mean':0.6,'sigma':0.1}],['gaussmf',{'mean':0.8,'sigma':0.1}]],
    [['gaussmf',{'mean':0.2,'sigma':0.1}],['gaussmf',{'mean':0.4,'sigma':0.1}],['gaussmf',{'mean':0.6,'sigma':0.1}],['gaussmf',{'mean':0.8,'sigma':0.1}]],
    [['gaussmf',{'mean':0.2,'sigma':0.1}],['gaussmf',{'mean':0.4,'sigma':0.1}],['gaussmf',{'mean':0.6,'sigma':0.1}],['gaussmf',{'mean':0.8,'sigma':0.1}]]
    ]
mf["gauss_3"] = [
    [['gaussmf',{'mean':0.2,'sigma':0.1}],['gaussmf',{'mean':0.2,'sigma':0.1}],['gaussmf',{'mean':0.2,'sigma':0.1}]],
    [['gaussmf',{'mean':0.2,'sigma':0.1}],['gaussmf',{'mean':0.2,'sigma':0.1}],['gaussmf',{'mean':0.2,'sigma':0.1}]],
    [['gaussmf',{'mean':0.2,'sigma':0.1}],['gaussmf',{'mean':0.2,'sigma':0.1}],['gaussmf',{'mean':0.2,'sigma':0.1}]]
    ]
mf["gbell_4"] = [
    [['gbellmf',{'a':4,'b':2,'c':0.1}],['gbellmf',{'a':4,'b':2,'c':0.6}],['gbellmf',{'a':4,'b':2,'c':0.8}],['gbellmf',{'a':4,'b':2,'c':1}]],
    [['gbellmf',{'a':4,'b':2,'c':0.1}],['gbellmf',{'a':4,'b':2,'c':0.6}],['gbellmf',{'a':4,'b':2,'c':0.8}],['gbellmf',{'a':4,'b':2,'c':1}]],
    [['gbellmf',{'a':4,'b':2,'c':0.1}],['gbellmf',{'a':4,'b':2,'c':0.6}],['gbellmf',{'a':4,'b':2,'c':0.8}],['gbellmf',{'a':4,'b':2,'c':1}]],
    ]

bell_mf_structure = "Bell b1 b2 b3"
gauss_mf_structure = "Gaussian b1 b2"
output_mf_structure = "Linear c1 c2 c3 c4"
tplFile = os.getcwd() + "/pfuzzy_tpl.fll"

experiments = []
experiments.append({'name':"gauss_4mf_5_epoch_0.001L",'num_input':3, 'num_output':8, 'mf_type':'Gaussian', 'tpl_file': tplFile, 'output_struct': output_mf_structure, 'mf_struct':gauss_mf_structure,'mf':mf["gauss_4"],'epochs':5,'alpha':0.001})
experiments.append({'name':"gauss_4mf_20_epoch_0.001L",'num_input':3, 'num_output':8, 'mf_type':'Gaussian', 'tpl_file': tplFile, 'output_struct': output_mf_structure, 'mf_struct':gauss_mf_structure,'mf':mf["gauss_4"],'epochs':20,'alpha':0.001})
experiments.append({'name':"gauss_4mf_25_epoch_0.001L",'num_input':3, 'num_output':8, 'mf_type':'Gaussian', 'tpl_file': tplFile, 'output_struct': output_mf_structure, 'mf_struct':gauss_mf_structure,'mf':mf["gauss_4"],'epochs':25,'alpha':0.001})
experiments.append({'name':"gauss_4mf_25_epoch_0.01L",'num_input':3, 'num_output':8, 'mf_type':'Gaussian', 'tpl_file': tplFile, 'output_struct': output_mf_structure, 'mf_struct':gauss_mf_structure,'mf':mf["gauss_4"],'epochs':25,'alpha':0.01})
experiments.append({'name':"gauss_4mf_25_epoch_0.0001L",'num_input':3, 'num_output':8, 'mf_type':'Gaussian', 'tpl_file': tplFile, 'output_struct': output_mf_structure, 'mf_struct':gauss_mf_structure,'mf':mf["gauss_4"],'epochs':25,'alpha':0.0001})
experiments.append({'name':"gauss_3mf_25_epoch_0.001L",'num_input':3, 'num_output':8, 'mf_type':'Gaussian', 'tpl_file': tplFile, 'output_struct': output_mf_structure, 'mf_struct':gauss_mf_structure,'mf':mf["gauss_3"],'epochs':25,'alpha':0.001})
experiments.append({'name':"gbell_4mf_25_epoch_0.001L",'num_input':3, 'num_output':8, 'mf_type':'Bell', 'tpl_file': tplFile, 'output_struct': output_mf_structure, 'mf_struct':bell_mf_structure,'mf':mf["gbell_4"],'epochs':25,'alpha':0.001})




In [22]:
from sklearn.model_selection import train_test_split
X_train, X_final_test, Y_train, Y_final_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [23]:
%matplotlib inline
from sklearn.model_selection import KFold
kf = KFold(n_splits=10, shuffle=True)
kf.get_n_splits(X_train)

count = 1
test_data_splits = []
for train_index, test_index in kf.split(X):
    test_data_splits.append({"train_index": train_index, "validation_index": test_index})




In [24]:
import time
for idx, experiment in enumerate(experiments):
    print("Experiment Id " + str(idx))
    print("Experiment Name " + experiment["name"])
    mfc = membershipfunction.MemFuncs(experiment["mf"])
    test_run_results = []
    test_split = test_data_splits[idx]
    X_train, X_test = X[test_split["train_index"]], X[test_split["validation_index"]]
    Y_train, Y_test = Y[test_split["train_index"]], Y[test_split["validation_index"]]
    experiment["test_split"] = test_split
    # Train the ANFIS and collect the prediction
    start = time.time()
    anf = anfis.ANFIS(X_train, Y_train, mfc)
    anf.trainHybridJangOffLine(epochs=experiment["epochs"])
    experiment["training_time"] = time.time() - start
    experiment["training_error"] = anf.getTrainingError()
    experiment["consequents"] = anf.consequents
    experiment["mf"] = anf.memFuncs




0.00019952 ... -0.0001998  -0.00014912
  -0.00033545]
 [ 0.00054349  0.00031242  0.00018516 ... -0.0001986  -0.00015402
  -0.00030927]]
(221, 8)
current error: 0.12481351314513266
current avg error: 0.12481351314513266
convergence: False
convergence 2: False
convergence 3: False
convergence 4: False
convergence 5: False
convergence 6: False
epoch 11
epoch 11
convergence: False
consequents
[[-9.01028702e-04 -8.59701627e-04 -7.84379566e-04 ... -3.23837436e-04
  -6.43414029e-05 -1.38235029e-03]
 [-1.51063216e-02 -1.07413956e-02 -8.50767310e-03 ... -1.59243448e-03
  -2.71951777e-03  2.15585201e-03]
 [-1.91825601e-03 -2.27441077e-03 -2.52532689e-03 ... -1.46675930e-03
  -4.09244431e-04 -6.13359289e-03]
 ...
 [-1.49969079e-02 -1.06415522e-02 -8.41467439e-03 ... -1.57164415e-03
  -2.70940987e-03  2.15664008e-03]
 [-1.92440817e-03 -2.27784476e-03 -2.52330012e-03 ... -1.42850684e-03
  -3.84215431e-04 -6.06442473e-03]
 [-5.17078493e-04 -4.83454204e-04 -4.63548211e-04 ... -3.41586132e-04
  -3.280

In [28]:
from fuzzylite import FllImporter
import re
import random

def text_replace(text, items_to_replace):
    rep = dict((re.escape(k), v) for k, v in items_to_replace.items()) 
    pattern = re.compile("|".join(items_to_replace.keys()))  
    return pattern.sub(lambda m: items_to_replace[re.escape(m.group(0))], text)

def prep_input_mf(num_input, mf_arr_all_inputs, mf_struct, mf_type):
    input_mf = []
    for ni in range(num_input):
        mf_str = ""
        mf_arr = mf_arr_all_inputs[ni]
        num_input_mf = len(mf_arr)
        for mf in range(num_input_mf):
            mf_values = mf_arr[mf][1]
            text_tpl = "term: in"+ str(ni+1) + "mf" + str(mf+1) + " " + mf_struct
            if mf_type == 'Gaussian':
                params = {"b1": str(mf_values["mean"]), "b2": str(mf_values["sigma"])}
            if mf_type == 'Bell':
                params = {"b1": str(mf_values["a"]), "b2": str(mf_values["b"]), "b3": str(mf_values["c"])}                
            if mf != 0:
                mf_str = mf_str + "\n" + "\t" 
            mf_str = mf_str + text_replace(text_tpl, params)
            mf_str = '' + mf_str + ''   
        input_mf.append(mf_str)
    return input_mf

def prep_output(num_output, num_output_mf, output_struct, consequents):
    out_str = ""
    for ni in range(num_output):
        mf_str = 'OutputVariable: out'+ str(ni+1)
        mf_str = (mf_str + "\n" 
        + "\t" + "enabled: true" + "\n"
        + "\t" + "range: -10.000 10.000" + "\n"
        + "\t" + "lock-range: false" + "\n"
        + "\t" + "aggregation: none" + "\n"
        + "\t" + "defuzzifier: WeightedAverage TakagiSugeno" + "\n"
        + "\t" + "default: nan" + "\n"
        + "\t" + "lock-previous: false")
        for mf in range(num_output_mf):
            text_tpl = "term: out"+ str(ni+1) + "mf" + str(mf+1) + " " + output_struct
            output_params = {"c1": str(consequents[mf*4, ni]), "c2": str(consequents[(mf*4) + 1, ni]), "c3": str(consequents[(mf*4) + 2, ni]), "c4": str(consequents[(mf*4) + 3, ni])}
            mf_str = mf_str + "\n" + "\t" 
            mf_str = mf_str + text_replace(text_tpl, output_params)
            mf_str = '' + mf_str + ''   
        if(ni == 0):
            out_str = mf_str
        else:
            out_str = out_str + "\n" + mf_str
    return out_str

def prep_rules(num_input_mf, num_output):
    rules_str = ""
    rule_num = 1
    for in1_mf in range(num_input_mf):
        for in2_mf in range(num_input_mf):
            for in3_mf in range(num_input_mf):
                rule_str = "rule: if in1 is in1mf" + str(in1_mf+1) + " and in2 is in2mf" + str(in2_mf+1) + " and in3 is in3mf" + str(in3_mf+1) 
                out_count = 1
                for ni in range(num_output):
                    if out_count == 1:
                        rule_str = rule_str + " then out1 is out1mf" + str(rule_num)
                    else:
                        rule_str = rule_str + " and out" + str(out_count) + " is out" + str(out_count) + "mf" + str(rule_num)
                    out_count = out_count + 1
                if rule_num == 1:
                    rules_str = rule_str
                else:
                    rules_str = rules_str + "\n" + "\t" + rule_str
                rule_num = rule_num + 1
                rules_str = '' + rules_str + ''
    return rules_str

def write_fill_file(tplFile, runFile, input_mf, out_str, rules_str):
    fin = open(tplFile, "rt")
    data = fin.read()
    #replace all occurrences of the required string
    data = data.replace('IN1', input_mf[0])
    data = data.replace('IN2', input_mf[1])
    data = data.replace('IN3', input_mf[2])
    data = data.replace('OUT', out_str)
    data = data.replace('RULES', rules_str)

    #close the input file
    fin.close()

    #open the input file in write mode
    fin = open(runFile, "wt")
    #overrite the input file with the resulting data
    fin.write(data)
    #close the file
    fin.close()

def init_fuzzy_engine(runFile):
    importer = FllImporter()
    with open(runFile, "r") as myfile:
        data_to_read = "".join(myfile.readlines()[1:])
    return importer.engine(data_to_read)

def prep_sample_consequents(num_output, num_output_mf):
    no_of_consequents_per_output = 4 * num_output_mf
    consequents = np.zeros(shape=(no_of_consequents_per_output,num_output))
    for row in range(no_of_consequents_per_output):
        consequents_row = []
        for col in range(num_output):
            consequents[row, col] = random.uniform(-1, 1)   
    return consequents

def setup_takagi_sugeno(experiment):
    runFileDir = "/tmp"
    mf_arr_all_inputs = experiment["mf"]
    mf_arr = mf_arr_all_inputs[0]
    num_input_mf = len(mf_arr)
    num_output_mf = 1
    num_output_mf = pow(num_input_mf, 3)
    input_mf = prep_input_mf(experiment["num_input"], mf_arr_all_inputs, experiment["mf_struct"], experiment["mf_type"])
    # if "consequents" not in experiment:
    #     experiment["consequents"] = prep_sample_consequents(num_output, num_output_mf)

    out_str = prep_output(experiment["num_output"], num_output_mf, experiment["output_struct"], experiment["consequents"])
    rules_str = prep_rules(num_input_mf, experiment["num_output"])
    runFile = runFileDir + "/" + experiment["name"] + ".fll"
    write_fill_file(experiment["tpl_file"], runFile, input_mf, out_str, rules_str)
    fuzzy_engine = init_fuzzy_engine(runFile)
    return fuzzy_engine

def predict_fuzzy(fuzzy_engine, X, num_output):
    predict_delta_y_arr = []
    for idx, s in enumerate(X):
        in1 = fuzzy_engine.variable("in1")
        in1.value = s[0]
        in2 = fuzzy_engine.variable("in2")
        in2.value = s[1]
        in3 = fuzzy_engine.variable("in3")
        in3.value = s[2]
        fuzzy_engine.process()
        pred = []
        for out_idx in range(num_output):
            out = fuzzy_engine.variable("out" + str(out_idx+1))
            if(math.isnan(out.value)):
                pred.append(0)
            else:
                pred.append(out.value)
        predict_delta_y_arr.append(pred)
    return predict_delta_y_arr    

def calc_error(predict_Y, actual_Y):
    return np.linalg.norm(predict_Y-actual_Y)
    # return list(np.array(predict_Y) - np.array(actual_Y))

def calc_total_error(errors):
    return sum(sum(abs(x)) for x in errors)

#del experiments[7:len(experiments)]
experiments.append({'name':"untrained_gauss_4mf_25_epoch_0.001L",'num_input':3, 'num_output':8, 'mf_type':'Gaussian', 'tpl_file': tplFile, 'output_struct': output_mf_structure, 'mf_struct':gauss_mf_structure,'mf':mf["gauss_4"],'epochs':25,'alpha':0.001, 'consequents': prep_sample_consequents(8, 64), 'test_split': experiments[0]['test_split'],'training_time': 'untrained', 'training_error': 'untrained'})
 #print(experiments[8]["name"], experiments[8].keys())

for idx, experiment in enumerate(experiments):
    fuzzy_engine = setup_takagi_sugeno(experiment)
    test_split = experiment['test_split']
    X_train, X_test = X[test_split["train_index"]], X[test_split["validation_index"]]
    Y_train, Y_test = Y[test_split["train_index"]], Y[test_split["validation_index"]]
    start = time.time()
    experiment["predict_Y"] = predict_fuzzy(fuzzy_engine, X_test, experiment["num_output"])
    experiment["validation_avg_error"] = calc_error(experiment["predict_Y"], Y_test)/len(X_test)
    # print(errors) 
    experiment["validation_time"] = time.time() - start
    # experiment["validation_total_error"]  = calc_total_error(experiment["validation_errors"])
    # experiment["validation_avg_error"]  = (experiment["validation_total_error"]/experiment["num_output"])/len(X_test)
    print(experiment["name"], "dataset_size", len(X_test), "training_time = ", experiment["training_time"], "training_error = ", experiment["training_error"], "validation_time = ", experiment["validation_time"], "validation_avg_error = ",experiment["validation_avg_error"] )



gauss_4mf_5_epoch_0.001L dataset_size 25 training_time =  188.44060802459717 training_error =  0.0005625597997331523 validation_time =  0.020447731018066406 validation_avg_error =  0.002007944498759833
gauss_4mf_20_epoch_0.001L dataset_size 25 training_time =  755.2922449111938 training_error =  0.000579822914697459 validation_time =  0.020289897918701172 validation_avg_error =  0.0018067944145270572
gauss_4mf_25_epoch_0.001L dataset_size 25 training_time =  888.0249390602112 training_error =  0.0005675537323681082 validation_time =  0.020348072052001953 validation_avg_error =  0.0018560552967668478
gauss_4mf_25_epoch_0.01L dataset_size 25 training_time =  895.2552120685577 training_error =  0.0005761992988882565 validation_time =  0.01837778091430664 validation_avg_error =  0.0012567005278994888
gauss_4mf_25_epoch_0.0001L dataset_size 25 training_time =  995.0412862300873 training_error =  0.0005378908714089157 validation_time =  0.028252840042114258 validation_avg_error =  0.00249660

In [29]:
final_test = []
# 'untrained_gauss_4mf_25_epoch_0.001L' index is 7
final_test.append(experiments[7])
# 'untrained_gauss_4mf_25_epoch_0.001L' index is 7
final_test.append(experiments[1])
for idx, experiment in enumerate(final_test):
    fuzzy_engine = setup_takagi_sugeno(experiment)
    start = time.time()
    experiment["predict_Y"] = predict_fuzzy(fuzzy_engine, X_final_test, experiment["num_output"])
    # print(predict_Y)
    # experiment["validation_errors"] = calc_error(experiment["predict_Y"], Y_final_test)
    experiment["validation_avg_error"] = calc_error(experiment["predict_Y"], Y_final_test)/len(X_final_test)
    # print(errors)
    experiment["validation_time"] = time.time() - start
    # experiment["validation_total_error"]  = calc_total_error(experiment["validation_errors"])
    # experiment["validation_avg_error"]  = (experiment["validation_total_error"]/experiment["num_output"])/len(X_final_test)
    print(experiment["name"], "dataset_size", len(X_final_test), "training_time = ", experiment["training_time"], "training_error = ", experiment["training_error"], "validation_time = ", experiment["validation_time"], "validation_avg_error = ",experiment["validation_avg_error"] )

untrained_gauss_4mf_25_epoch_0.001L dataset_size 49 training_time =  untrained training_error =  untrained validation_time =  0.04001307487487793 validation_avg_error =  0.26629263004526055
gauss_4mf_20_epoch_0.001L dataset_size 49 training_time =  755.2922449111938 training_error =  0.000579822914697459 validation_time =  0.03723621368408203 validation_avg_error =  0.0015246519147056937


In [31]:
all_data_test = []
# 'untrained_gauss_4mf_25_epoch_0.001L' index is 7
all_data_test.append(experiments[7])
# 'untrained_gauss_4mf_25_epoch_0.001L' index is 7
all_data_test.append(experiments[1])
for idx, experiment in enumerate(all_data_test):
    fuzzy_engine = setup_takagi_sugeno(experiment)
    start = time.time()
    experiment["predict_Y"] = predict_fuzzy(fuzzy_engine, X, experiment["num_output"])
    # print(predict_Y)
    # experiment["validation_errors"] = calc_error(experiment["predict_Y"], Y)
    experiment["validation_avg_error"] = calc_error(experiment["predict_Y"], Y)/len(X)
    # print(errors)
    experiment["validation_time"] = time.time() - start
    # experiment["validation_total_error"]  = calc_total_error(experiment["validation_errors"])
    # experiment["validation_avg_error"]  = (experiment["validation_total_error"]/experiment["num_output"])/len(X)
    print(experiment["name"], "dataset_size", len(X), "training_time = ", experiment["training_time"], "training_error = ", experiment["training_error"], "validation_time = ", experiment["validation_time"], "validation_avg_error = ",experiment["validation_avg_error"] )


untrained_gauss_4mf_25_epoch_0.001L dataset_size 245 training_time =  untrained training_error =  untrained validation_time =  0.1981041431427002 validation_avg_error =  0.11742006267425435
gauss_4mf_20_epoch_0.001L dataset_size 245 training_time =  755.2922449111938 training_error =  0.000579822914697459 validation_time =  0.19237089157104492 validation_avg_error =  0.0008166139680807904


In [ ]:
import random
%matplotlib inline

num_of_outputs = 3
fig = plt.figure(figsize=(16,12))
plt.title('Predicted yield against the actual')
cm = plt.get_cmap('gist_rainbow')

ax = fig.add_subplot(111)
NUM_COLORS = 16
colors = [cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)]
random.shuffle(colors)
ax.set_prop_cycle(color=colors)

selected_yields_arr = df_selected_yields.to_numpy()

for yield_no in range(num_of_outputs, 4):
    ax.plot(range(len(selected_yields_arr)),selected_yields_arr[:, yield_no], label='original-'  + df_selected_yields.columns[yield_no])


def plot_predications(ax, label, num_of_outputs, df_selected_yields, predict_delta_y_arr):
    predict_yields = np.zeros(df_selected_yields.shape)

    for yield_no in range(7):
        predict_yields[0, yield_no] = df_selected_yields.iloc[0, yield_no]

    for idx, predict_delta_y in enumerate(predict_delta_y_arr):
        for yield_no in range(7):
            predict_yields[idx+1, yield_no] = selected_yields_arr[idx, yield_no] + predict_delta_y[yield_no]

    for yield_no in range(num_of_outputs, 4):
        ax.plot(range(len(predict_yields)), predict_yields[:, yield_no], label= label + '-' + df_selected_yields.columns[yield_no])

plot_predications(ax, 'ANFIS Trained', num_of_outputs, df_selected_yields, experiments[1]["predict_Y"])
plot_predications(ax, 'untrained', num_of_outputs, df_selected_yields, experiments[7]["predict_Y"])

plt.legend(loc='upper left')
plt.savefig("yeild_pred_compare_5yr.png")


In [74]:
import numpy as np
import matplotlib.pyplot as plt
from skfuzzy import gaussmf, gbellmf, sigmf

def maximum(a, b, c): 
  
    if (a >= b) and (a >= c): 
        largest = a 
  
    elif (b >= a) and (b >= c): 
        largest = b 
    else: 
        largest = c 
          
    return largest 

def gaussmf(x, mean, sigma):
    """
    Gaussian fuzzy membership function.

    Parameters
    ----------
    x : 1d array or iterable
        Independent variable.
    mean : float
        Gaussian parameter for center (mean) value.
    sigma : float
        Gaussian parameter for standard deviation.

    Returns
    -------
    y : 1d array
        Gaussian membership function for x.
    """
    return np.exp(-((x - mean) ** 2.) / float(sigma) ** 2.)

def plotMF(plt, x, mean, sigma, plot_color):
    y = gaussmf(x, mean, sigma)
    plt.plot(x,y, plot_color)

start_x = 0
end_x = 1.2
input_range = np.arange(start_x, end_x, 0.001)

### PC 1
input_value = 0.6
mf1_y = gaussmf(input_value,0.20571977564581453,0.0491192242633381)
mf2_y = gaussmf(input_value,0.5466377875234245,0.05447030341262251)
mf3_y = gaussmf(input_value,0.8000000105491593,0.0999999576463103)
max_y = maximum(mf1_y, mf2_y, mf3_y)

fig = plt.figure(figsize=(16,12))
plotMF(plt, input_range, 0.20571977564581453,0.0491192242633381, 'r')
plotMF(plt, input_range, 0.5466377875234245,0.05447030341262251, 'g')
plotMF(plt, input_range, 0.8000000105491593,0.0999999576463103, 'b')
plt.axvline(input_value, 0, max_y, label='MF1', c='y')
plt.axhline(mf2_y, start_x, input_value/end_x, label='pyplot horizontal line', c='y')
plt.axhline(mf3_y, start_x, input_value/end_x, label='pyplot horizontal line', c='y')
plt.legend(labels=["PCA1 MF1", "PCA1 MF2", "PCA1 MF3"], fontsize=16)
plt.savefig("pc1_mf" + ".png")

fig.clear()


### PC 2
input_value = 0.55
mf1_y = gaussmf(input_value,0.22042538992326025, 0.061544754167856645)
mf2_y = gaussmf(input_value,0.3998348205882725, 0.1006452825975365)
mf3_y = gaussmf(input_value,0.5999999999999686, 0.10000000000015556)
max_y = maximum(mf1_y, mf2_y, mf3_y)

fig = plt.figure(figsize=(16,12))
plotMF(plt, input_range, 0.22042538992326025, 0.061544754167856645, 'r')
plotMF(plt, input_range, 0.3998348205882725, 0.1006452825975365, 'g')
plotMF(plt, input_range, 0.5999999999999686, 0.10000000000015556, 'b')
plt.axvline(input_value, 0, max_y, label='MF1', c='y')
plt.axhline(mf1_y, start_x, input_value/end_x, label='pyplot horizontal line', c='y')
plt.axhline(mf2_y, start_x, input_value/end_x, label='pyplot horizontal line', c='y')
plt.axhline(mf3_y, start_x, input_value/end_x, label='pyplot horizontal line', c='y')
plt.legend(labels=["PCA2 MF1", "PCA2 MF2", "PCA2 MF3"], fontsize=16)
plt.savefig("pc2_mf" + ".png")

fig.clear()

### Delta PC1
start_x = -0.5
end_x = 1.2
input_range = np.arange(start_x, end_x, 0.001)
input_value = 0.2
mf1_y = gaussmf(input_value, -0.1375461431080145, 0.3258052465358045)
mf2_y = gaussmf(input_value,0.39837064971028435, 0.10566987668499336)
mf3_y = gaussmf(input_value,0.6000000000012675, 0.09999999999309502)
max_y = maximum(mf1_y, mf2_y, mf3_y)

fig = plt.figure(figsize=(16,12))
plotMF(plt, input_range, -0.1375461431080145, 0.3258052465358045, 'r')
plotMF(plt, input_range, 0.39837064971028435, 0.10566987668499336, 'g')
plotMF(plt, input_range, 0.6000000000012675, 0.09999999999309502, 'b')
plt.axvline(input_value, 0, max_y, label='MF1', c='y')
plt.axhline(mf1_y, start_x, input_value*2.1, label='pyplot horizontal line', c='y')
plt.axhline(mf2_y, start_x, input_value*2.1, label='pyplot horizontal line', c='y')
plt.axhline(mf3_y, start_x, input_value*2.1, label='pyplot horizontal line', c='y')
plt.legend(labels=["Delta PCA1 MF1", "Delta PCA1 MF2", "Delta PCA1 MF3"], fontsize=16)
plt.savefig("deltapc1_mf" + ".png")

fig.clear()


<Figure size 1152x864 with 0 Axes>

<Figure size 1152x864 with 0 Axes>

<Figure size 1152x864 with 0 Axes>